In [33]:
pip install nltk

In [34]:
import random
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
# Load data from bot.json
with open('bot.json', 'r') as file:
    data = json.load(file)

In [36]:
# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Initialize lists for storing words, classes, and training data
words = []
classes = []
documents = []
ignore_words = ['?', '!']

In [37]:
# Preprocess data, tokenize words, lemmatize, create bag of words
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize words
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents
        documents.append((w, intent['tag']))
        # Add unique classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [38]:
# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# Sort classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)


18 documents
6 classes ['cultural_events', 'cultural_practices', 'heritage_conservation', 'heritage_sites', 'historical_influence', 'local_cuisine']
65 unique lemmatized words ["'s", '.', 'about', 'and', 'any', 'are', 'area', 'being', 'can', 'celebrated', 'celebration', 'conservation', 'conserve', 'cuisine', 'cultural', 'culture', 'custom', 'delicacy', 'delta', 'dish', 'effort', 'event', 'explain', 'festival', 'followed', 'from', 'ha', 'heritage', 'historical', 'history', 'how', 'impact', 'in', 'influence', 'initiative', 'is', 'landmark', 'local', 'made', 'me', 'nadu', 'observed', 'of', 'on', 'or', 'place', 'practice', 'preserved', 'recommend', 'region', 'ritual', 'shaped', 'site', 'some', 'tamil', 'tell', 'that', 'the', 'there', 'to', 'tradition', 'traditional', 'unique', 'what', 'you']


In [39]:
# Save words and classes to pickle files for later use
import pickle
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [40]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Initialize bag of words
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Separate features and labels
train_x = np.array([sample[0] for sample in training])
train_y = np.array([sample[1] for sample in training])

# Define and compile the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

# Train the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               8448      
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 6)                 390       
                                                                 
Total params: 17094 (66.77 KB)
Trainable params: 17094 (66.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
4/4 [=

In [41]:
# Save the model
model.save('delta_model.h5')
print("Model created and saved successfully!")

Model created and saved successfully!
